In [163]:
import random
import time

class SimplifiedYannakakis:
    def __init__(self, relations):
        self.relations = relations

    def perform_semi_join(self, r1, r2):
        join_attribute_r1 = {t[1] for t in r1}
        join_attribute_r2 = {t[0] for t in r2}
        semi_join_result = join_attribute_r1.intersection(join_attribute_r2)
        return [t for t in r1 if t[1] in semi_join_result]

    def reduce_relations(self):
        for i in reversed(range(len(self.relations) - 1)):
            self.relations[i] = self.perform_semi_join(self.relations[i], self.relations[i + 1])

    def execute_join(self):
        self.reduce_relations()
        join_result = self.relations[0]
        for i in range(1, len(self.relations)):
            next_relation = self.relations[i]
            join_result = [(t1[0],) + t1[1:] + t2[1:] for t1 in join_result for t2 in next_relation if t1[-1] == t2[0]]
        return join_result

def hash_relation(relation, join_attribute_index):
    hash_map = {}
    for tuple_ in relation:
        key = tuple_[join_attribute_index]
        hash_map.setdefault(key, []).append(tuple_)
    return hash_map

def line_join(relations):
    join_result = relations[0]
    for i in range(1, len(relations)):
        hash_map = hash_relation(relations[i], 0)
        new_join_result = []
        for tuple_ in join_result:
            join_key = tuple_[-1]
            for join_tuple in hash_map.get(join_key, []):
                new_join_result.append(tuple_ + join_tuple[1:])
        join_result = new_join_result
    return join_result

# Functions to generate the datasets with common join attributes
def generate_common_values(num_commons, max_random_value):
    return random.sample(range(1, max_random_value + 1), num_commons)

def generate_relation_R1_R2_with_commons(num_tuples, max_random_value, common_values):
    relation_R1 = [(i, random.choice(common_values)) for i in range(1, num_tuples + 1)]
    relation_R2 = [(random.choice(common_values), i) for i in range(1, num_tuples + 1)]
    return relation_R1, relation_R2

def generate_relation_R3(num_tuples):
    return [(i, i) for i in range(1, num_tuples + 1)]

common_values = generate_common_values(20, 5000)

# Generate the relations with the common join attributes
R1, R2 = generate_relation_R1_R2_with_commons(100, 5000, common_values)
R3 = generate_relation_R3(100)

yannakakis = SimplifiedYannakakis([R1, R2, R3])

start_time_yannakakis = time.time()
yannakakis_result = yannakakis.execute_join()
end_time_yannakakis = time.time()
execution_time_yannakakis = end_time_yannakakis - start_time_yannakakis

start_time_line_join = time.time()
line_join_result = line_join([R1, R2, R3])
end_time_line_join = time.time()
execution_time_line_join = end_time_line_join - start_time_line_join
results_same = yannakakis_result == line_join_result

print(f"Yannakakis Execution Time: {execution_time_yannakakis} seconds")
print(f"Line Join Execution Time: {execution_time_line_join} seconds")
print(f"Are results the same: {results_same}")
print(f"First 10 Results from Yannakakis: {yannakakis_result[:10]}")
print(f"First 10 Results from Line Join: {line_join_result[:10]}")

Yannakakis Execution Time: 0.0071980953216552734 seconds
Line Join Execution Time: 0.001062154769897461 seconds
Are results the same: True
First 10 Results from Yannakakis: [(1, 2328, 29, 29), (1, 2328, 52, 52), (1, 2328, 67, 67), (2, 2328, 29, 29), (2, 2328, 52, 52), (2, 2328, 67, 67), (3, 2415, 8, 8), (3, 2415, 10, 10), (3, 2415, 36, 36), (4, 4688, 18, 18)]
First 10 Results from Line Join: [(1, 2328, 29, 29), (1, 2328, 52, 52), (1, 2328, 67, 67), (2, 2328, 29, 29), (2, 2328, 52, 52), (2, 2328, 67, 67), (3, 2415, 8, 8), (3, 2415, 10, 10), (3, 2415, 36, 36), (4, 4688, 18, 18)]
